In [1]:
# links

# acs
# https://api.census.gov/data/2018/acs/acs5/profile/examples.html
# https://api.census.gov/data/2018/acs/acs5/profile/geography.html
# https://api.census.gov/data/2018/acs/acs5/variables.html
# https://data.census.gov/cedsci/table?q=United%20States
# https://www.census.gov/data/developers/data-sets/acs-5year.html

# qwi
# https://www.census.gov/data/developers/data-sets/qwi.html
# https://api.census.gov/data/timeseries/qwi/sa/variables.html
# https://api.census.gov/data/timeseries/qwi/sa/examples.html
# https://api.census.gov/data/timeseries/qwi.html
# https://lehd.ces.census.gov/doc/QWIPU_Data_Schema.pdf
# https://lehd.ces.census.gov/doc/QWI_101.pdf

In [6]:
%pwd

'C:\\Users\\Tola\\Documents\\Github\\capstone'

In [7]:
%cd Documents\Github\capstone

[WinError 3] The system cannot find the path specified: 'Documents\\Github\\capstone'
C:\Users\Tola\Documents\Github\capstone


In [8]:
#manipulate dataframes/geodataframes in python
import pandas as pd
import geopandas as gpd

#make API calls with python
import requests

#allows us to store results of API call cleanly
import json

#for parsing the urls

In [9]:
import numpy as np
from shapely.geometry import Point
from fiona.crs import from_epsg

In [10]:
from ipyleaflet import Map, GeoData, GeoJSON, basemaps, basemap_to_tiles, Icon, Circle, Marker, LayerGroup, WidgetControl
from ipywidgets import Button 
import ipywidgets as widgets

In [22]:
#drawing basic map
center = (40.7210907,-73.9877836)
basemap = basemap_to_tiles(basemaps.CartoDB.Positron)

icon = Icon(icon_url='icon.png', icon_size=[15, 15])
marker = Marker(location=center, draggable=False, icon=icon, opacity = 0)

m = Map(layers=(basemap, ), center=center, zoom=15, min_zoom = 11, max_zoom = 20)

###HALF-MI BUFFER
#set location of center as location of the point
markerlocation=marker.location
    
radius = 402 #in-meters
color = "black"
fill_color = "white"
fill_opacity = .2
weight = 2

##click on map to add marker, removes old marker as well
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        layer_group.clear_layers();
        marker = Marker(location=kwargs.get('coordinates'), draggable=False, icon=icon, opacity = 0)
        layer_group.add_layer(marker)
        
        #updating the circle's location
        markerlocation=marker.location
        circle.location = markerlocation
        layer_group.add_layer(circle);

        ########################### adding the black overlay
        df2 = pd.DataFrame(markerlocation)

        df=df2.transpose()
        df.columns=['Latitude','Longitude']

        #creating a geodataframe from the point
        gdf = gpd.GeoDataFrame(
            df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
        #creating circle for cut
        # gdf.crs = 'EPSG:4326'
        gdf.crs = from_epsg(3857)
        test=gdf.copy()
        #making buffer
        test['geometry'] = test.geometry.buffer(.004,  cap_style=1, join_style=1)
        # change back
        test.crs = from_epsg(3857)
        #creating rectangle for cut
        rect_orig=gdf.copy()
        rect_orig['geometry'] = rect_orig.geometry.buffer(4,  cap_style=1, join_style=1)
        #difference
        diff = gpd.overlay(rect_orig, test, how='difference')

        geo_data = GeoData(geo_dataframe = diff,
                       style={'fillColor': 'black', 'color':'black', 'weight':2,'fillOpacity':0.5},
    #                    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Boundary')
        layer_group.add_layer(geo_data)
        ################
        
m.on_interaction(handle_click)
######

#create half-mi buffer
circle = Circle()
circle.location = markerlocation
circle.radius = radius
circle.color = color
circle.fill_color = fill_color
circle.fill_opacity = fill_opacity
circle.weight = weight
circle.opacity = 0

###########

    
# def on_location_changed(event):
# #remove any circles existing first
#     layer_group.clear_layers();

# #update location of center as location of the point
#     markerlocation = marker.location
#     circle.location = markerlocation
#     layer_group.add_layer(circle);
        
# pass
# marker.observe(on_location_changed, 'location')
# #######


#adding button that performs the analysis on click 
button = Button(description="Analyze")

def on_button_clicked(b):   
    #extracting the marker location for the buffer
    location=circle.location
    
    df2 = pd.DataFrame(location)

    df=df2.transpose()
    df.columns=['Latitude','Longitude']

    #creating a geodataframe from the point
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
#creating circle for cut
    # gdf.crs = 'EPSG:4326'
    gdf.crs = from_epsg(3857)
    test=gdf.copy()
    #making buffer
    test['geometry'] = test.geometry.buffer(.004,  cap_style=1, join_style=1)
    # change back
    test.crs = from_epsg(3857)
#creating rectangle for cut
    rect_orig=gdf.copy()
    rect_orig['geometry'] = rect_orig.geometry.buffer(4,  cap_style=1, join_style=1)
#difference
    diff = gpd.overlay(rect_orig, test, how='difference')

    geo_data = GeoData(geo_dataframe = diff,
                   style={'fillColor': 'black', 'color':'black', 'weight':2,'fillOpacity':0.5},
#                    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Boundary')
    layer_group.add_layer(geo_data)
    
button.on_click(on_button_clicked)

    
widget_control = WidgetControl(widget=button, position='topright')
m.add_control(widget_control)


layer_group = LayerGroup(layers=(marker, circle))
m.add_layer(layer_group)

display(m)

Map(center=[40.7210907, -73.9877836], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

In [30]:
#Extracting latitude and longitude values as separate strings from marker location ie from center
lat = str(markerlocation[0])
lon = str(markerlocation[1])
print(lat,'+', lon)

40.7210907 + -73.9877836


In [35]:
#Figure out which state and census block a latitude/longitude is in
#USING PARAMETERS 

#Encode parameters 
# lat = '42.3295'
# lon = '-71.0826'

#Contruct request URL
url = f'https://geo.fcc.gov/api/census/block/find?'

# api call using variables instead of directly is way easier
data_url = f'{url}lat={lat}&lon={lon}&format=json'
#Get response from API
response=requests.get(data_url)

#Parse json in response
data = response.json()

print(response.text)

#Print FIPS code
print(data['results'][0]['state_fips'])
print(data['results'][0]['county_fips'])
print(data['results'][0]['county_name'])

{"input":{"lat":40.7210907,"lon":-73.9877836},"results":[{"block_fips":"360610030012000","bbox":[-73.988354,40.719833,-73.98695,40.721234],"county_fips":"36061","county_name":"New York","state_fips":"36","state_code":"NY","state_name":"New York","block_pop_2015":504,"amt":"AMT001","bea":"BEA010","bta":"BTA321","cma":"CMA001","eag":"EAG001","ivm":"IVM001","mea":"MEA002","mta":"MTA001","pea":"PEA001","rea":"REA001","rpc":"RPC001","vpc":"VPC001"},{"block_fips":"360610030012001","bbox":[-73.989114,40.720077,-73.987769,40.721462],"county_fips":"36061","county_name":"New York","state_fips":"36","state_code":"NY","state_name":"New York","block_pop_2015":531,"amt":"AMT001","bea":"BEA010","bta":"BTA321","cma":"CMA001","eag":"EAG001","ivm":"IVM001","mea":"MEA002","mta":"MTA001","pea":"PEA001","rea":"REA001","rpc":"RPC001","vpc":"VPC001"}]}
36
36061
New York


In [38]:
#Extracting the county and state being studied 
state = data['results'][0]['state_fips']
county_fips = data['results'][0]['county_fips']

print(state)
print(county_fips)

36
36061


In [41]:
#Separating out the county fips from the state
county = county_fips[2:]

print(county)

061


In [55]:
#Downloading census 

# api_key = 9330dc4bf086a84f19fb412bb15f232507301de6
with open('census_key.txt') as key:
    api_key=key.read().strip()
 
# here is defining the variables...
# state='36' #state will be defined outside of here
# county='005,047,061' #as will county...should i bother putting county there?  
# tract='001002'
time='2019-Q2'

MSA_short='metropolitan%20statistical%20area/micropolitan%20statistical%20area'
acs_url = f'https://api.census.gov/data/2018/acs/acs5/'
qwi_url = f'https://api.census.gov/data/timeseries/qwi/sa'

In [56]:
# https://api.census.gov/data/2018/acs/acs5/profile?get=DP04_0001E&for=tract:*&in=state:36%20county:005,047&key=9330dc4bf086a84f19fb412bb15f232507301de6
NAME_acs='B01003_001E'

# api call using variables instead of directly is way easier
acs_data_url = f'{acs_url}?&get={NAME_acs}&for=tract:*&in=state:{state}%20county:{county}&key={api_key}'
response_acs=requests.get(acs_data_url)
# print(response_acs.text)

#load the response into a JSON
data_acs=response_acs.json()
#ignoring the first element which is column names, replacing columns with the columns from the table
acs=pd.DataFrame(data_acs[1:], columns=data_acs[0])

acs.head()

,B01003_001E,state,county,tract
0,2579,36,061,003700
1,3022,36,061,008601
2,16684,36,061,004400
3,4313,36,061,008603
4,0,36,061,008602


In [62]:
# 	https://api.census.gov/data/timeseries/qwi/sa?get=year,agegrp,quarter&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&in=state:24&time=2010-Q1&key=YOUR_KEY_GOES_HERE
NAME_qwi='geography,industry,sex'

# api call using variables instead of directly is way easier
qwi_data_url = f'{qwi_url}?&get={NAME_qwi}&for={MSA_short}:*&in=state:{state}&time={time}&key={api_key}'
response_qwi=requests.get(qwi_data_url)
# print(response_qwi.text)

#load the response into a JSON
data_qwi=response_qwi.json()
#ignoring the first element which is just column names, replacing columns with the columns from the table
qwi=pd.DataFrame(data_qwi[1:], columns=data_qwi[0])

qwi.head()

,geography,industry,sex,time,state,metropolitan statistical area/micropolitan statistical area
0,3610580,1111,0,2019-Q2,36,10580
1,3610580,1111,1,2019-Q2,36,10580
2,3610580,1111,2,2019-Q2,36,10580
3,3610580,1112,0,2019-Q2,36,10580
4,3610580,1112,1,2019-Q2,36,10580


In [ ]:
qwi: geography,industry,sex,agegrp,education,firmsize

acs:
population, sex: B01003_001E,B01001_002E,B01001_026E, 
education: B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E
english or no: B16001_001E,B16001_002E
household income: B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E, B19001_017E
median rent as percentage of income: B25071_001E
means of transportation to work: B08301_001E, B08301_002E, B08301_003E, B08301_003E, B08301_010E, B08301_016E, B08301_017E, B08301_018E, B08301_019E, B08301_021E
travel time to work: B08303_002E, B08303_003E, B08303_004E, B08303_005E, B08303_006E, B08303_007E, B08303_008E, B08303_009E, B08303_010E, B08303_011E, B08303_012E, B08303_013E
household type: B11001_002E, B11001_007E

In [ ]:
#proportional split
